In [1]:
import itertools


In [2]:
def get_pdf(var, cond, cpds):
    """
    Returns conditional probability P(var|cond) given in cpds.

    var -- a tuple (name, value)
    cond -- a dictionary of variable values {n1:v1, n2:v2, ...}
    cpds -- a dictionary of conditional probability distributions(cpd)
   
    The key of a single cpd in dictionary is the variable that cpd is for. 

    The value of a single cpf is a pair: (parents, probabilities). The order of
    probabilities is defined by truth values of parents starting with all
    variables set to False (as is usual in truth tables). 
    Therefore, having 2 conditional variables, the order is:
    
    False False
    False True
    True False
    True True
    
    Example network:
    Having a small network X1->X2->X3, where P(X1)=0.7, 
    P(X2|X1)=0.6, P(X2|not X1)=0.3, P(X3|X2)=0.8, and P(X3|notX2)=0.1, the 
    encoding would be: 

    cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6],
        'X3': (('X2', ), [0.1, 0.8],
    }

    Examples:
    >>> get_pdf(('X1', True), {}, cpds_small))
    0.7
    >>> get_pdf(('X2', True), {'X1':True}, cpds_small))
    0.6
    >>> get_pdf(('X3', False), {'X2':False}, cpds_small))
    0.9
    """

    if var[0] not in cpds:
        raise Exception('Variable {} does not exist'.format(var[0]))

    parents, probs = cpds[var[0]]
    # create value vector
    values = [cond[cvar] for cvar in parents]

    # index 
    index = sum(2**vi * v for vi, v in enumerate(values[::-1]))
    return probs[index] if var[1] else 1-probs[index]



In [3]:
cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6]),
        'X3': (('X2', ), [0.1, 0.8]),
    }

get_pdf(('X3', True), {'X2':False, 'X1':True}, cpds_small)

0.1

In [4]:
def jpd(variables, cpds):
    """
    Returns a joint probability distribution  of all variables from 
    the cpds. Joint probability distribution is calculated using the 'chain rule'. 

    The joint probability distribution should be returned as a list having 
    $2**len(variables)$ probabilities summing to 1 (if the sum is not 1, 
    you are doing something wrong). 

    The order of probabilities should be the same as in cpds.
    For example, with three variables, the correct order of variable values is:

    False, False, False
    False, False, True
    False, True, False
    False, True, True
    True, False, False
    True, False, True
    True, True, False
    True, True, True

    Hint: use module 'itertools', specifically the 'product' method.  

    Example:
    Consider the same small network X1->X2->X3 with cpds:
    cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6],
        'X3': (('X2', ), [0.1, 0.8],
    }

    >>> jpd(['X1','X2','X3'], cpds_small)
    [0.18900000000000003, 0.021000000000000005, 0.018, 0.07200000000000001, 
    0.252, 0.027999999999999997, 0.0839999999999998, 0.336]

    First value explanation:
    P(X1=false,X2=false,X3=false) = P(X1=false) * P(X2=false|X1=false) * P(X3=false|X2=false) =
                                  = 0.3 * 0.7 * 0.9 = 0.189

    """

    # YOUR CODE HERE
    per = list(itertools.product([False, True], repeat=len(variables)))
    prob = []
    
    for i in range(len(per)):
        res = 1
        for j in range(len(variables)):
            cond = {}
            parents, probs = cpds[variables[j]]
            for p in parents:
                cond[p] = per[i][variables.index(p)]
            res *= get_pdf((variables[j], per[i][j]), cond, cpds)
        prob.append(res)
    return prob

In [5]:
cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6]),
        'X3': (('X2', ), [0.1, 0.8]),
}
jpd(['X1','X2','X3'], cpds_small)

[0.18900000000000003,
 0.021000000000000005,
 0.018,
 0.07200000000000001,
 0.252,
 0.027999999999999997,
 0.08399999999999998,
 0.336]

In [6]:
def marginalize(jpd, variables, values):
    """ 
    Returns probability P(values) by marginalizing out the variables that 
    are not in values. 

    jpd -- is a joint probability distribution as described in the jpd method. 
    variables -- all variables of the bayesian network.
    values -- is a list of (name, value) pairs.

    Examples (using the same distribution as before):
    >>> j = jpd(['X1','X2','X3'], cpds_small)
    >>> marginalize(j, ['X1', 'X2', 'X3'], [('X1', True), ('X2', False)])
    0.28
    >>> marginalize(j, ['X1', 'X2', 'X3'], [('X1', True), ('X2', False), ('X3', True)])
    0.028

    """
    
    # YOUR CODE HERE
    per = list(itertools.product([False, True], repeat=len(variables)))
    ls = []
    for v in values:
        ls.append((variables.index(v[0]), v[1]))
    
    res = 0
    for p in per:
        eq = True
        for l in ls:
            if p[l[0]] != l[1]:
                eq = False
                break
        if eq == True:
            res += jpd[per.index(p)]
    return res


In [7]:
cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6]),
        'X3': (('X2', ), [0.1, 0.8]),
}
j = jpd(['X1','X2','X3'], cpds_small)
print(marginalize(j, ['X1', 'X2', 'X3'], [('X1', True), ('X2', False)]))
print(marginalize(j, ['X1', 'X2', 'X3'], [('X1', True), ('X2', False), ('X3', True)]))

0.28
0.027999999999999997


In [8]:
def cond_prob(jpd, variables, values1, values2):
    """
    Returns P(values1|values2)

    Examples:
    >>> cond_prob(j, ['X1','X2','X3'], [('X2',True)], [('X1',True)]) #P(X2|X1)
    0.6
    >>> cond_prob(j, ['X1','X2','X3'], [('X1',True)], [('X2',True)]) #P(X1|X2)
    0.8235...
    >>> cond_prob(j, ['X1','X2','X3'], [('X1',True),('X2',False)], [('X2',True)]) #P(X1,notX2|X2)
    0.0
    """

    # YOUR CODE HERE
    return marginalize(jpd, variables, values1+values2) / marginalize(jpd, variables, values2)


In [9]:
cpds_small = {
        'X1': (tuple(), [0.7]),
        'X2': (('X1', ), [0.3, 0.6]),
        'X3': (('X2', ), [0.1, 0.8]),
}
j = jpd(['X1','X2','X3'], cpds_small)
print(cond_prob(j, ['X1','X2','X3'], [('X2',True)], [('X1',True)]))
print(cond_prob(j, ['X1','X2','X3'], [('X1',True)], [('X2',True)]))
print(cond_prob(j, ['X1','X2','X3'], [('X1',True),('X2',False)], [('X2',True)]))

0.6
0.8235294117647058
0.0


In [10]:
variables = ['S','C','R','E','G','M','F']
cpds_company = {
 'S': (tuple(), [0.05]),
 'C': (tuple(), [0.8]),
 'R': (('S', ), [0.1, 0.5]),
 'E': (('S', 'C'), [0.3, 0.01, 0.7, 0.1]),
 'G': (('R', 'E'), [0.9, 0.4, 0.4, 0.1]),
 'M': (('G', ), [0.2, 0.4]),
 'F': (('M', 'G'), [0.4, 0.1, 0.9, 0.8]),
}
j = jpd(variables, cpds_company)
print(cond_prob(j, variables, [('F',True)], [('S',True)]))
print(cond_prob(j, variables, [('E',True)], [('R',True), ('F',True)]))

#cpds_company['C']

0.43256
0.10675727484774253
